In [1]:
import json

In [2]:
with open("./data/nsynth-train.jsonwav/nsynth-train/examples.json") as f:
    rawData = json.load(f)

In [3]:
data = []

for d in rawData:
    if (rawData[d]['instrument_family'] in [0, 1, 3, 4, 7, 8] and rawData[d]['instrument_source_str'] == 'acoustic'):
        data.append({"audioFile" : d, "pitch" : rawData[d]['pitch']})

len(data)

64950

In [4]:
import librosa
# from matplotlib import figure
import matplotlib.pyplot as plt
import numpy as np
import random
from pydub import AudioSegment
from tqdm import tqdm
import os
import glob
from multiprocessing import Process

In [5]:
data[312]["audioFile"].split("-")[1].removeprefix('0')

'107'

In [6]:
a = [21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108]
# inside of data/train, make a folder for each value of a
for i in a:
    os.makedirs(f'./data/nsynth-train.jsonwav/nsynth-train/train/{i}', exist_ok=True)

In [7]:
def convertToSpectrogram(data):
    for i in range(len(data)):
        audio = AudioSegment.from_wav("./data/nsynth-train.jsonwav/nsynth-train/audio/" + data[i]["audioFile"]+".wav")
        cropped_audio = audio[:3000 + random.randint(-2000, 1000)]
        cropped_audio.export("./data/nsynth-train.jsonwav/nsynth-train/temp/" + data[i]["audioFile"]+".wav", format="wav")
    
    for i in tqdm(range(len(data))):
        array, _ = librosa.load("./data/nsynth-train.jsonwav/nsynth-train/temp/" + data[i]["audioFile"]+".wav")
        D = librosa.stft(array)
        S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)
        plt.figure().set_figwidth(12)
        plt.axis('off')
        librosa.display.specshow(S_db, x_axis="time", y_axis="hz")
        plt.savefig(f"./data/nsynth-train.jsonwav/nsynth-train/train/{data[i]["pitch"]}/{data[i]["audioFile"]}.png", bbox_inches='tight', pad_inches=0)
    
    files = glob.glob("./data/nsynth-train.jsonwav/nsynth-train/temp/*")
    for f in files:
        os.remove(f)


In [8]:
for i in tqdm(range(0, 640, 2)):
    p1 = Process(target=convertToSpectrogram, args=([data[i*100:(i+1)*100]]))
    p1.start()
    p1.join()


100%|██████████| 320/320 [33:39<00:00,  6.31s/it]
